# Jupyter Notebook of test_runner


In [1]:
import numpy as np
from model_loader import load_pretrained_model, load_dataloader
from augmentations import DataAugmentationTesting
from embeddings_similarity_module import EmbeddingSimilarityModule
#from cam_module import ClassActivationMapping
from collections import defaultdict
import logging
import random
import json
import os


# Create a logger object
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)  # Set the minimum level of messages to log

# Create a file handler that logs debug and higher level messages
file_handler = logging.FileHandler('example.log', mode='w')
file_handler.setLevel(logging.DEBUG)
file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(file_formatter)

# Create a console handler that logs debug and higher level messages
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
console_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(console_formatter)

# Add both handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)

def load_config(config_path):
    with open(config_path, 'r') as config_file:
        return json.load(config_file)

def save_test_results(results):
    # Implement the logic to save results to a file or database
    pass

def print_summary(results):
    # Summarize test results
    print("Test Summary:")
    for result in results:
        print(f"Index: {result['index']}, Model Output: {result['output']}, Embedding Similarity: {result['embedding_similarity']}, CAM Result: {result['cam_result']}")

#***************************************************      Main           *************************************************************
if __name__ == "__main__":

    config_path = "config.json"
    config = load_config(config_path)
    model = load_pretrained_model(config)
    logging.info("Model Loaded")
    dataloader = load_dataloader(config)
    logging.info("Dataloader Loaded")

    # Initialize the Data Augmentation Module
    augmentor_sandbox = DataAugmentationTesting(model, dataloader, config)
    
    # # Initialize processing modules
    embedding_similarity_module = EmbeddingSimilarityModule(model, dataloader, config)

    #features, labels =embedding_similarity_module.extract_features()
    #embedding_similarity_module.plot_tsne(features,labels)
    # cam = ClassActivationMapping(model)

    # # Generate test index range
    # class_indices = defaultdict(list)
    # for idx, (_, label) in enumerate(dataloader.dataset):
    #     class_indices[int(label)].append(idx)
    # print(class_indices)
    # N = 3  # Number of samples per class
    # test_index_range = []
    # for indices in class_indices.values():
    #     if len(indices) >= N:
    #         test_index_range.extend(random.sample(indices, N))
    #     else:
    #         test_index_range.extend(indices)

    # # Shuffle the test index range to ensure random order
    # random.shuffle(test_index_range)

    # Prepare to collect results
    test_index_range = [0]
    test_results = []

    # Run tests on the specific range of indexes
    for index in test_index_range:
        # Augment the sample
        augmentor_sandbox.replace_with_noise(index)

        # Get the augmented sample
        aug_sample, aug_label = augmentor_sandbox.get_augmented_sample(index)

        # Run model inference on augmented sample
        model_output = augmentor_sandbox.run_model_on_augmented_sample(aug_sample)

        # Compute embedding similarity
        es_features, es_label = embedding_similarity_module.extract_features(aug_sample, aug_label)
        es_result = 0 #TODO compute distance metric for similarity
        # embedding_similarity_module.plot_tsne(es_features, es_label)

        # # Compute class activation mapping
        # cam_result = cam.compute(augmented_sample, model_output)
        cam_result = None
        # Store results
        test_results.append({
            "index": index,
            "output": model_output,
            "embedding_similarity": es_result,
            "cam_result": cam_result
        })

    # # # Optionally, save and summarize results
    # # save_test_results(test_results)
    # print_summary(test_results)



Baseline_CNN1D(
  (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (conv1): Conv1d(2, 64, kernel_size=(7,), stride=(1,))
  (relu1): ReLU()
  (maxpool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
  (relu2): ReLU()
  (maxpool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (feature_extractor): ModuleList(
    (0): Conv1d(2, 64, kernel_size=(7,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=16000, out_features=256, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=256, out_features=4, bias=True)
  (logSoftmax): LogSoftmax(dim=1)
) cpu
cpu
(array([[ 0.83196074, -0.65662307,  0.32264539, ..., -0.6